In [23]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
import re
import pickle

In [6]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rupes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rupes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rupes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rupes\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rupes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
df = pd.read_csv("News _dataset/Fake.csv")

In [8]:
df2 = pd.read_csv("News _dataset/True.csv")

In [9]:
df['label'] = 0
df2['label'] = 1

In [10]:
len(df), len(df2)

(23481, 21417)

In [11]:
dataset = pd.concat([df, df2], axis = 0).reset_index(drop = True)

In [12]:
len(dataset)

44898

In [13]:
dataset.sample(10)

,title,text,subject,date,label
43129,Sudan's Bashir visits Darfur ahead of U.S. san...,KHARTOUM (Reuters) - Sudanese President Omar a...,worldnews,"September 19, 2017",1
11482,WIKILEAKS REMINDS THE WORLD: “Obama has a hist...,Wikileaks founder Julian Assange reminded us i...,politics,"Mar 5, 2017",0
35454,United States calls for delivery of U.N. aid s...,WASHINGTON (Reuters) - The United States is pr...,worldnews,"December 18, 2017",1
21950,Was Gaddafi Right About JFK?,The late Libyan leader Muammar Gaddafi remarke...,US_News,"November 6, 2017",0
41260,"Yemen Islamist party members arrested, ratchet...",ADEN (Reuters) - Security forces in the southe...,worldnews,"October 11, 2017",1
2671,Judge Who Blocked Trump’s Muslim Ban REPEATED...,While Trump is whining up a storm on Twitter a...,News,"February 5, 2017",0
8177,‘Sex And The City’ Star Forced To Do CRINGEWO...,Everyone knows actress Kristin Davis from her ...,News,"February 11, 2016",0
20346,OBAMA’S GAL PAL Loretta Lynch WON’T RECUSE HER...,Reasonable Americans expected never expected u...,left-news,"Jul 1, 2016",0
20345,"LIBERAL HUFFINGTON POST HEADLINE: “Man, Bill C...",Is it possible that Bill s latest move could v...,left-news,"Jul 1, 2016",0
42051,Barca closed soccer stadium to show support fo...,BARCELONA (Reuters) - Barcelona president Jose...,worldnews,"October 1, 2017",1


In [14]:
dataset = shuffle(dataset, random_state = 42).reset_index(drop = True)

In [15]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocessing(text):
    text = text.lower()

    text = re.sub(r'http\S+|www\S+', '', text)

    words = nltk.word_tokenize(text)

    words = [word for word in words if word not in stop_words]

    lemmatized = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(lemmatized)

In [16]:
dataset['processed_text'] = dataset['text'].apply(preprocessing)

In [17]:
dataset['processed_title'] = dataset['title'].apply(preprocessing)

In [18]:
dataset.to_csv("processed_dataset.csv", index = False) 

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [20]:
vectorizer = TfidfVectorizer(max_features = 5000)
X = vectorizer.fit_transform(dataset['processed_text'])
y = dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [21]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

svm_model = LinearSVC()
svm_model.fit(X_train, y_train)

y_svm_pred = svm_model.predict(X_test)
print(accuracy_score(y_svm_pred, y_test))

0.9947661469933184


In [ ]:
with open("News_Detection_SVM.pkl", "wb") as f:
    pickle.dump(svm_model, f)

In [22]:
import xgboost as xgb

xgb_clf = xgb.XGBClassifier(eval_metric='logloss')
xgb_clf.fit(X_train, y_train)

y_pred_xgb = xgb_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_xgb))

0.9979955456570155


In [ ]:
with open("News_Detection_XGB.pkl", "wb") as f:
    pickle.dump(xgb_clf, f)